In [3]:
%matplotlib inline

import os

import keras as keras
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, Activation, add, Add, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.datasets import cifar10
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

random_state = 42

Using TensorFlow backend.


# Lesson2 畳み込みニューラルネットワーク(CNN)

## Section1 解説

### 1.1 CNN基礎
**ConvolutioalNeuralNetwork**(**CNN**)は、画像認識、音声認識,自然言語処理などにおいて幅広く使用されている。

Lesson1のMLPは全結合(Dense)層のみで構成されていたが、CNNは**Convolution(畳み込み)層**と**Pooling(プーリング)層**を組み合わせて構築していく。(出力に近い層は全結合層を組み合わせることも多く、全結合層も使用する)

CNNにおいて重要なのは、
- 局所結合・重み共有による**パラメータ削減**
- **不変性**

であり、それぞれ畳み込み層・プーリング層に対応する。

#### 1.1.1 局所結合によるパラメータ削減

2次元の画像では、近いところにある画素同士の関係性が強いのに対して、離れた場所にある画素同士の関係性は薄いと考えられる。

CNNの畳み込み層では、近い画素同士の結合のみを考えることでパラメータを削減している。

下図に全結合層のイメージを示す。入力のすべてのユニットは出力のすべてのユニットと結合を持っていることが分かる。(下図のパラメータ数は5×5=25)

![全結合層image](https://github.com/matsuolab-edu/dl4us/raw/b34f8964fe3cdbbf0dd527dc381926721538ac8c/lesson2/figures/sparse_1.png)

これに対し、CNNの畳み込み層では出力のユニットは**入力のある一定範囲に対してのみ結合**を持つ。下図にCNNの畳み込み層のイメージを示す。パラメータ数が13となっていて、この例では約半分にできていることが分かる。

![CNNの畳み込み層image](https://github.com/matsuolab-edu/dl4us/raw/b34f8964fe3cdbbf0dd527dc381926721538ac8c/lesson2/figures/sparse_2.png)

全結合層では画素同士の位置関係の情報を排除(ベクトルに落とし込んでいた)していたのに対し、畳み込み層では画素同士の情報を保持したまま扱うことで、結合を疎にすることを可能にしている。

#### 1.1.2 重み共有によるパラメータ削減

ある画像がネコであるかそうでないかを判別するときに、画像内のどこにネコがいるかはどうでもよい。

全結合層では層間で1つのパラメータは1度だけ使われるが、畳み込み層では入力のすべての位置で同じパラメータを使用する。(重み共有)

下図に全結合層でのパラメータ使用のイメージを示す。1つの重み(黒い矢印)は特定のユニット間でのみ使用される。

![全結合層パラメータimage](https://github.com/matsuolab-edu/dl4us/raw/b34f8964fe3cdbbf0dd527dc381926721538ac8c/lesson2/figures/sharing_2.png)

下図に畳み込み層でのパラメータ使用のイメージを示す。畳み込み層では1つの重み(黒い矢印)はすべての場所で使用される。

![畳み込み層パラメータimage](https://github.com/matsuolab-edu/dl4us/raw/b34f8964fe3cdbbf0dd527dc381926721538ac8c/lesson2/figures/sharing_1.png)

畳み込み層ではどこにあるかという情報を捨象し、なにがあるかのみを残すことでパラメータを大幅に削減している。入力画像が大きくなってもパラメータ数は増えない。

使用するパラメータ数を抑えることで、学習を効率的に進めることができる。

ただし、どこにあるのかという位置情報が重要な場合は、画像内の位置によって異なるパラメータを使用する場合がある。

#### 1.1.3 不変性

プーリング層では、獲得した特徴のゆがみやズレに対しての頑強性をあげるため、小さな領域での統計量(Max, Mean)などを取る。また、画像サイズを小さくする役割もある。

![プーリング層](http://cs231n.github.io/assets/cnn/maxpool.jpeg)

**ネットワークの構成**

基本的には、

畳み込み層→プーリング層→畳み込み層→プーリング層→・・・

というように畳み込みとプーリングを繰り返してく。全結合層は位置情報を失うため、ネットワークの最後でのみ使用する。

最初の方の層で局所的な特徴(エッジ等)を抽出し、層が進むにつれ大局的な特徴(タイヤ等)を抽出することができる。

![階層的な概念の抽出](https://github.com/matsuolab-edu/dl4us/raw/b34f8964fe3cdbbf0dd527dc381926721538ac8c/lesson2/figures/cnn_feature_extraction.png)

このような**階層的な概念の抽出**が深層学習の大きな特徴である。

以下でそれぞれの層を具体的に説明していく。

### 1.2 Convolution(畳み込み)層
#### 1.2.1 畳み込み層の考え方
畳み込み層における畳み込みとは、入力に対してフィルターをかけた(畳みこんだ)ときに得られる値のこと。

#### 1.2.2 2次元入力に対する畳み込み
畳み込み計算は下図のように行う。
![畳み込み計算](https://github.com/matsuolab-edu/dl4us/raw/b34f8964fe3cdbbf0dd527dc381926721538ac8c/lesson2/figures/conv.png)

畳み込み層ではある領域においてのフィルターに対する類似度のようなものを計算している。
フィルターは１つの特徴に対応するので、複数のフィルターを設定することでより複数の特徴を獲得していると考えられる。
フィルターのサイズを大きくすると広い範囲の特徴を、小さくすると小さな範囲の特徴を獲得することが出来る。

#### 1.2.3 3次元入力に対する畳み込み
CNNでの各層の入力は実際には(縦のピクセル数)×(横のピクセル数)×(フィルター数)の3階テンソルとなっている。それに合わせて、フィルターも3階テンソルとなっているが、畳み込みの考え方自体は2次元と同様。

#### 1.2.4 パラメータ削減の例
全結合層を畳み込み層で置き換えることで実際にどれくらいのパラメータを削減できるかを見てみる。

入力画像が10×10×3(合計300ピクセル)の場合

全結合層でユニット数300の隠れ層につなぐ場合、パラメータ数は300×300×300=90300となる。

畳み込み層で考えると、5×5×3のフィルターを100枚用いた場合は、5×5×3×100=7600となり、全結合層のパラメータ数の約1/12。

また、全結合層では入力画像のサイズに比例してパラメータ数が増えるが、畳み込み層では増えないのでパラメータ削減の効果は入力画像が大きくなるにつれ大きくなる。

#### 1.2.5 出力のサイズ
入力の縦or横の次元数を$N$、フィルタの縦or横の次元数を$F$、フィルタを動かす幅を$S$とすると、出力のサイズは以下の様に計算できる。

$$
(N-F)/S+1
$$

#### 1.2.6 パディング
1.2.2の畳み込み計算の図を見ると、出力のサイズ(2, 3)は入力のサイズ(3, 4)よりも少し小さくなっている。このように、入力に対してそのまま畳み込みを行うと特徴マップのサイズは縮小する。

下図では、16次元の入力に対してサイズ6のフィルタ－で畳み込みを行っている。畳み込みのたびに特徴マップのサイズが縮小しているので、3層までしか進むことが出来ない。

![16次元の入力に対するサイズ6の畳み込み](https://github.com/matsuolab-edu/dl4us/raw/b34f8964fe3cdbbf0dd527dc381926721538ac8c/lesson2/figures/pool_1.png)

特徴マップが縮小してしまうのを防ぐために入力の両端に対して0などの値をくっつける。これを**パディング**という。下図では左端に3つ、右端に3つずつユニットを追加している。

![パディング](https://github.com/matsuolab-edu/dl4us/raw/b34f8964fe3cdbbf0dd527dc381926721538ac8c/lesson2/figures/pool_2.png)

パディングをしないと、特徴マップの端の方のユニットは中央よりも畳み込みされる回数が少ないので、情報として過少に評価されていると考えられる。パディングにより、端の方のユニットに対する畳み込みの回数が増えるので、端の方に重要な情報がある場合には有効だと考えることが出来る。

何もくっつけないパディング(パディングしない、出力のサイズが小さくなる)
を**Valid**、入力と出力のサイズが変わらないようにするパディングを**Same**という。

#### 1.2.7 Kerasによる実装
Kerasで畳み込み層を設定するには、keras.layers.Conv2Dを使用する。

主な引数
- filters : フィルター(カーネル)の数
- kernel_size : フィルターの大きさ
- strides : フィルターを動かす幅
- padding : パディング
- activation : 活性化関数
- use_bias : バイアス項の有無

In [6]:
from IPython.display import display, HTML

HTML('''<div style="display: flex; justify-content: row;">
    <img src="https://i.giphy.com/media/6EjTPebp1oWxG/giphy.gif">
</div>''')


上のgifと同じ操作を行うコードをサンプルとしていかに示す。

In [7]:
# サンプル画像 (5x5)
sample_image = np.array([[1, 1, 1, 0, 0],
                         [0, 1, 1, 1, 0],
                         [0, 0, 1, 1, 1],
                         [0, 0, 1, 1, 0],
                         [0, 1, 1, 0, 0]]
                        ).astype('float32').reshape(1, 5, 5, 1)

# フィルタ
W = np.array([[1, 0, 1],
              [0, 1, 0],
              [1, 0, 1]]).astype('float32').reshape(3, 3, 1, 1)

model = Sequential()

model.add(Conv2D(1, kernel_size=(3, 3), strides=(1, 1),
                 padding='valid', input_shape=(5, 5, 1), use_bias=False))
model.layers[0].set_weights([W])

model.predict(sample_image).reshape(3, 3)

array([[4., 3., 4.],
       [2., 4., 3.],
       [2., 3., 4.]], dtype=float32)

### 1.3 Pooling(プーリング)層
#### 1.3.1 プーリングの考え方
小さな領域に対して統計量(Max, Mean)を取ることで、一のズレなどに対して頑強な特徴抽出を行う。

#### 1.3.2 Kerasによる実装
Kerasで畳み込み層を設定するにはMaxPooling2D、AveragePooling2D、GlobalMaxPooling2D、GlobalAveragePooling2Dを使用する。

主な引数
- pool_size : プーリングする領域のサイズ
- strides : ウィンドウを動かすサイズ
- padding : パディング

![pooling_keras](http://cs231n.github.io/assets/cnn/maxpool.jpeg)

上図と同じ操作を行うコードを以下にサンプルとして示す。


In [8]:
# サンプル画像
sample_image = np.array([[1, 1, 2, 4],
                         [5, 6, 7, 8],
                         [3, 2, 1, 0],
                         [1, 2, 3, 4]]
                        ).astype("float32").reshape(1, 4, 4, 1)

model = Sequential()

model.add(MaxPooling2D(pool_size=(2, 2), strides=None,
                       padding='valid', input_shape=(4, 4, 1)))

model.predict(sample_image).reshape(2, 2)

array([[6., 8.],
       [3., 4.]], dtype=float32)